In [ ]:
from analytics import sheets_api as sheets
from analytics import sheets_elements as elements
from analytics import api as ga
from analytics.entities import *
from constants import *
import pandas as pd
import gspread
import time

In [ ]:
# The name of this environment variable will need to be changed based on where your credentials are stored.
# The name must match the configured value in constants.py
%env ANALYTICS_CLIENT_SECRET_PATH=../../../do_not_commit_ga4_credentials.json

#### Authenticate and define parameters

In [ ]:
# Authentication
ga_authentication, drive_authentication, sheets_authentication = ga.authenticate(
    SECRET_NAME,
    ga.ga4_service_params,
    ga.drive_service_params,
    ga.sheets_service_params,
    port=OAUTH_PORT
)

REPORT_DATES = elements.get_bounds_for_month_and_prev(CURRENT_MONTH)
START_DATE_CURRENT = REPORT_DATES["start_current"]
END_DATE_CURRENT = REPORT_DATES["end_current"]
START_DATE_PRIOR = REPORT_DATES["start_previous"]
END_DATE_PRIOR = REPORT_DATES["end_previous"]

date_string = f"{START_DATE_CURRENT} - {END_DATE_CURRENT}"

default_params = {
    "service_system": ga_authentication,
    "start_date": START_DATE_CURRENT,
    "end_date": END_DATE_CURRENT,
}

hca_explorer_params = {
    **default_params,
    "property": ANALYTICS_PROPERTY_ID,
}

hca_explorer_params_from_analytics_start = {
    **hca_explorer_params,
    "start_date": ANALYTICS_START,
}
hca_explorer_params_with_custom_events = {
    **hca_explorer_params,
    "start_date": CUSTOM_EVENT_START,
}

date_args = [START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR]
# Events associataed with bulk and project exports
export_events = [
    EVENT_INDEX_FILE_MANIFEST_REQUESTED, EVENT_INDEX_FILE_MANIFEST_SELECTED,
    EVENT_INDEX_ANALYZE_IN_TERRA_REQUESTED, EVENT_INDEX_ANALYZE_IN_TERRA_SELECTED,
    EVENT_INDEX_BULK_DOWNLOAD_REQUESTED, EVENT_INDEX_BULK_DOWNLOAD_SELECTED,
]
# Misc custom events
misc_custom_events = [EVENT_HCA_EXPLORE_DATA_CLICKED, EVENT_SUPPORT_REQUEST_CREATED]

#### Get data from the Analytics api as Pandas DataFrames

In [ ]:
# Get the monthly traffic summary sheet, containing total users and pageviews for each month
# additional_data_path and additionl_detail_behavior are optional parameters that can be specified to add additional historical data
# additional_data_path must point to a json file in the same format as the users_over_time_history.json file in this folder
df_monthly_pageviews = elements.get_page_views_over_time_df(hca_explorer_params_from_analytics_start, additional_data_path=HISTORIC_UA_DATA_PATH, additional_data_behavior=elements.ADDITIONAL_DATA_BEHAVIOR.ADD)
# Get the number of pageviews and active users for each page during the current month
df_pageviews = elements.get_page_views_change(hca_explorer_params, *date_args)
# Get the number of clicks to each outbound link
df_outbound = elements.get_outbound_links_change(hca_explorer_params, *date_args)
# Get the number of sessions that start on a particular landing page
df_landing_pages = elements.get_landing_page_change(hca_explorer_params, *date_args)
# Get the number of visits to each tab on the explorer and the total number of users visiting.
# Includes users who visit the default tab
df_entity_selected = elements.get_index_entity_selected_change(hca_explorer_params, *date_args)
# Get the number of clicks and users for each pagination button
df_pagination = elements.get_index_entity_table_paginated_change(hca_explorer_params, *date_args)
# Get the number of clicks and users for each sort button
df_sort = elements.get_index_entity_table_sorted_change(hca_explorer_params, *date_args)
# Get the number of clicks and users for each filter name and value
df_filter = elements.get_index_filter_selected_change(hca_explorer_params, *date_args)
# Get the number of clicks and users for each file download button on the index page only
df_file_downloaded = elements.get_index_table_download_change(hca_explorer_params, *date_args)
# Get the number of times each export event was triggered
df_export = elements.get_event_count_over_time_df(hca_explorer_params_with_custom_events, export_events)
# Get the number of times each misc custom event was triggered
df_misc_custom_events = elements.get_event_count_over_time_df(hca_explorer_params_with_custom_events, misc_custom_events)

#### Save analytics data to sheets

In [ ]:
# Currently this example makes too many api queries, so it will fail. 
# This can be resolved by updating the package to use batch updating wherever possible for gspread-formatting
# and to add exception handling to other gspread calls so they retry after a certain amount of time.
# Ideally, it would be possible to reduce the number of api calls by batching all updates to the spreadsheet in one
# api call, but this does not appear to be possible with gspread
# See https://gspread-formatting.readthedocs.io/en/latest/#batch-mode-for-api-call-efficiency
# and https://developers.google.com/sheets/api/limits#exponential

dict_spreadsheet = {
    "Monthly Traffic Summary": df_monthly_pageviews,
    "Pageviews": df_pageviews,
    "Outbound Links": df_outbound,
    "Landing Pages": df_landing_pages,
    "Entity Selected": df_entity_selected,
    "Pagination": df_pagination,
    "Sort": df_sort,
    "Filter": df_filter,
    "File Downloaded": df_file_downloaded,
    "Exports": df_export,
    "Misc" : df_misc_custom_events,
}
sheet = sheets.create_sheet_in_folder(
        drive_authentication,
        SHEET_NAME,
        PARENT_FOLDER_NAME,
        override_behavior=sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE
    )
#TODO: need to update package to use batch for gspread_formatting wherever possible, otherwise this won't run
eventname_totalusers = {
    METRIC_EVENT_COUNT["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
    METRIC_TOTAL_USERS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
}

sheets.fill_spreadsheet_with_df_dict(
    sheet,
    dict_spreadsheet,
    sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE,
    
    column_formatting_options={
        "Monthly Traffic Summary": {
            DIMENSION_YEAR_MONTH["alias"]: sheets.COLUMN_FORMAT_OPTIONS.YEAR_MONTH_DATE,
            METRIC_ACTIVE_USERS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            METRIC_PAGE_VIEWS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },
        "Outbound Links": {
            SYNTHETIC_METRIC_CLICKS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            METRIC_TOTAL_USERS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },
        "Pageviews": {
            METRIC_PAGE_VIEWS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            METRIC_TOTAL_USERS["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },
        "Entity Selected": eventname_totalusers,
        "Pagination": eventname_totalusers,
        "Sort": eventname_totalusers,
        "Filter": eventname_totalusers,
        "File Downloaded": eventname_totalusers,
        "Exports": {event["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED for event in export_events},
        "Misc": {event["change_alias"]: sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED for event in misc_custom_events},
    },
    sheet_formatting_options={
        "Monthly Traffic Summary": {
            "extra_columns": 1,
             "extra_columns_width": 2000
        }
    },
    gspread_update_args={
        "Filter": {
            "value_input_option": gspread.utils.ValueInputOption.raw
        }
    }
)
monthly_traffic_worksheet = sheet.worksheet("Monthly Traffic Summary")
date_range = sheets.WorksheetRange(
    monthly_traffic_worksheet, 
    gspread.cell.Cell(1, 1), 
    gspread.cell.Cell(df_monthly_pageviews.index.size + 1, 2)
)
users_range = sheets.WorksheetRange(
    monthly_traffic_worksheet, 
    gspread.cell.Cell(1, 2), 
    gspread.cell.Cell(df_monthly_pageviews.index.size + 1, 3)
)
pageviews_range = sheets.WorksheetRange(
    monthly_traffic_worksheet, 
    gspread.cell.Cell(1, 3), 
    gspread.cell.Cell(df_monthly_pageviews.index.size + 1, 4)
)
time.sleep(45)
sheets.add_chart_to_sheet(
    sheets_authentication,
    sheet,
    sheet.worksheet("Monthly Traffic Summary"),
    sheets.CHART_TYPES.LINE,
    date_range,
    [users_range, pageviews_range],
    chart_position=gspread.cell.Cell(1, 6),
    chart_position_offset_x=75,
    chart_position_offset_y=75,
    title="Pageviews and Users Over Time"
)